In [29]:
import torch
import torch.nn as nn
import tiktoken

OKay, we already created the main pieces to create a gpt model from scratch.

## LLM architecture

The model architecture is very easy to understand, since many of its components are repeated.

In [30]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": True
}

In [31]:
#Let's create some useful dummy classes
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
    def forward(self, x):
        return x
        
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps = 1e-5):
        super().__init__()
    def forward(self, x):
        return x

In [32]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg)
            for _ in range(cfg["n_layers"])]
        )
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx) #Token embeddings
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device = in_idx.device)
        ) #Positional embeddings
        x = tok_embeds + pos_embeds #Add embeddings
        x = self.drop_emb(x) #Dropout
        x = self.trf_blocks(x) #Transformer blocks
        x = self.final_norm(x) #Layer Normalization
        logits = self.out_head(x) #Output (Linear layer)
        return logits
#And that's it! We have a complete GPT model.

In [33]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every days holds a"

In [34]:
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

In [35]:
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1528, 6622,  257]])


In [36]:
torch.manual_seed(123)

model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
logits.shape

torch.Size([2, 4, 50257])

Let's code the individual placeholders

In [37]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim=True)
        var = x.var(dim = -1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [38]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            nn.GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"])
        )
    def forward(self, x):
        return self.layers(x)

In [39]:
# This class implements a more efficient version of multi-head self-attention.
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out,
        context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        # Linear layer to project input to queries
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Linear layer to project input to keys
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Linear layer to project input to values
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Output projection after concatenating all heads
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        # Register a causal mask to prevent attending to future tokens
        #Again, this is a buffer, which means it's part of the model but doesn't need gradients.
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # Project input to keys, queries, and values
        #The output tensor shape is (batch, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        # Reshape for multi-head: (batch, tokens, heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Move heads dimension forward: (batch, heads, tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute attention scores: (batch, heads, tokens, tokens)
        attn_scores = queries @ keys.transpose(2, 3)
        # Create boolean mask for causal attention
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # Apply mask: set future positions to -inf
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # Softmax over last dimension to get attention weights
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        # Apply dropout to attention weights
        attn_weights = self.dropout(attn_weights)

        # Weighted sum of values
        context_vec = (attn_weights @ values).transpose(1, 2)
        # Concatenate all heads
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        # Final output projection
        context_vec = self.out_proj(context_vec)
        return context_vec

In [40]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            context_length = cfg["context_length"],
            num_heads = cfg["n_heads"],
            dropout = cfg["drop_rate"],
            qkv_bias = cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x


In [41]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg)
            for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx) #Token embeddings
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device = in_idx.device)
        ) #Positional embeddings
        x = tok_embeds + pos_embeds #Add embeddings
        x = self.drop_emb(x) #Dropout
        x = self.trf_blocks(x) #Transformer blocks
        x = self.final_norm(x) #Layer Normalization
        logits = self.out_head(x) #Output (Linear layer)
        return logits
        

In [43]:
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
display(out)
out.shape

tensor([[[-0.4390, -0.3360,  0.9460,  ...,  0.0993, -0.2259,  0.0505],
         [ 0.2534, -0.5048,  0.8330,  ...,  0.3176,  0.6749,  0.7259],
         [ 0.4626,  0.0613,  0.1522,  ...,  0.2537, -1.0529,  0.7553],
         [ 0.2351,  0.4379,  0.6051,  ...,  0.1011, -0.2638,  0.4092]],

        [[-0.6303, -0.6357,  0.6225,  ...,  0.1757, -0.3498, -0.0092],
         [-0.8555, -0.7012,  0.2259,  ...,  0.7143, -0.2225,  0.6269],
         [-0.2765, -1.2161,  0.7788,  ...,  0.1650, -1.6378,  0.9408],
         [ 0.2585, -0.5067,  0.2806,  ...,  0.2273, -0.9919,  1.0606]]],
       grad_fn=<UnsafeViewBackward0>)

torch.Size([2, 4, 50257])

In [45]:
total_params = sum(p.numel() for p in model.parameters())
f"{total_params:,}"

'163,037,184'

In [46]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim = -1)
        idx_next = torch.argmax(probas, dim = -1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim = 1)
    return idx

In [50]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
encoded_tensor

tensor([[15496,    11,   314,   716]])

In [52]:
model.eval()
out = generate_text_simple(
    model = model,
    idx = encoded_tensor,
    max_new_tokens=6,
    context_size=GPT_CONFIG_124M["context_length"]
)
tokenizer.decode(out.squeeze(0).tolist())

'Hello, I am negativity Mond progressives563 Lyon snippets'

THe model isn't trained yet, that's why he talks nonsenses